In [1]:
import os 


In [2]:
%pwd

'/Users/ravina/Desktop/deep_end_to_end/research'

In [3]:
os.chdir("../")

In [22]:
%pwd

'/Users/ravina/Desktop/deep_end_to_end'

In [23]:
#entity: return type of a function
from dataclasses import dataclass 
from pathlib import Path

@dataclass(frozen=True)  #this is not python class but dataclass, here you can define the veriables without using self keyword.
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path


In [24]:
#reading the yaml file

from kidneyDiseaseClassifier.constants import *
from kidneyDiseaseClassifier.utils.common import read_yaml,create_directories



In [25]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        

        create_directories([self.config.artifacts_root])
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.get('data_ingestion')
        if config is None:
            raise ValueError("Data Ingestion configuration is missing or invalid.")

        create_directories([config.get('root_dir')])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config


In [26]:
import os 
import zipfile
import gdown
from kidneyDiseaseClassifier import logger
from kidneyDiseaseClassifier.utils.common import get_size

In [27]:
! pip install --upgrade gdown



In [28]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self)-> str:
        """"fetch data from the url"""
        try: 
            dataset_url=self.config.source_URL
            zip_download_dir=self.config.local_data_file
            os.makedirs("artifacts/dat_ingestion",exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id=dataset_url.split("/")[-2]
            prefix="https://drive.google.com/uc?/export=download&id="
            gdown.download(prefix+file_id,zip_download_dir)
            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")
        
        
            
        except Exception as e:
            raise e 


    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [29]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-02-13 17:59:06,130:INFO:common:yaml file:config/config.yaml loaded successfully]
[2024-02-13 17:59:06,133:INFO:common:yaml file:params.yaml loaded successfully]
[2024-02-13 17:59:06,133:INFO:common:created directory at:artifacts]
[2024-02-13 17:59:06,134:INFO:common:created directory at:artifacts/data_ingestion]
[2024-02-13 17:59:06,134:INFO:3969266638:Downloading data from https://drive.google.com/file/d/15EoQXCKhMYBU8mjxnpVebni-JAZmtezo/view?usp=sharing into file artifacts/data_ingestion/data.zip]


FileURLRetrievalError: Failed to retrieve file url:

	Too many users have viewed or downloaded this file recently. Please
	try accessing the file again later. If the file you are trying to
	access is particularly large or is shared with many people, it may
	take up to 24 hours to be able to view or download the file. If you
	still can't access a file after 24 hours, contact your domain
	administrator.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?/export=download&id=15EoQXCKhMYBU8mjxnpVebni-JAZmtezo

but Gdown can't. Please check connections and permissions.